# Text Completions

Exploring text completions using llama-jax.

# Setup

In [1]:
from collections.abc import Sequence
from contextlib import contextmanager
from sys import stdout
from time import time_ns, perf_counter_ns as timer

import jax
from jax import numpy as jnp
from jax import random
from rich.live import Live
from rich.table import Table, Column

import llama_jax as ll

In [2]:
print(f"Available devices: {jax.devices()}")

Metal device set to: Apple M1 Max

systemMemory: 64.00 GB
maxCacheSize: 24.00 GB

Available devices: [METAL(id=0)]


W0000 00:00:1740515229.536891 4326682 mps_client.cc:510] WARNING: JAX Apple GPU support is experimental and not all JAX functionality is correctly supported!
I0000 00:00:1740515229.547092 4326682 service.cc:145] XLA service 0x161936a40 initialized for platform METAL (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1740515229.547100 4326682 service.cc:153]   StreamExecutor device (0): Metal, <undefined>
I0000 00:00:1740515229.548373 4326682 mps_client.cc:406] Using Simple allocator.
I0000 00:00:1740515229.548383 4326682 mps_client.cc:384] XLA backend will use up to 51539214336 bytes on device 0 for SimpleAllocator.


In [3]:
@contextmanager
def tps_report(n_tokens: int):
    start_time = timer()

    yield
        
    duration = (timer() - start_time) / 1000000000
    tps = n_tokens / duration
    
    print(f"Generated {n_tokens} tokens in {duration:0.1f} s ({tps:0.1f} tps)")


def render(content: str | Sequence[str]):
    if isinstance(content, str):
        content = [content]

    n = len(content)

    table = Table(show_header=False, show_edge=False, expand=True)
    
    for _ in range(n):
        table.add_column(Column(), ratio=1/n)

    table.add_row(*content)
    
    return table    

# Configure

In [4]:
# Parameters
max_tokens = 30
seed = time_ns()

In [5]:
# Configure generator
key = random.key(seed)
config = ll.checkpoint.load_config("Llama3.2-3B")
key, subkey = random.split(key)
generator = ll.text.generator(config, key=subkey, max_tokens=max_tokens)

# Warmup

In [6]:
prompt = "My name is Julien and I like to"

In [9]:
with tps_report(max_tokens):
    for token in generator(prompt):
        stdout.write(".")
    print()

..............................
Generated 30 tokens in 6.1 s (4.9 tps)


# Prompt 0: Trains

In [10]:
prompt = "I like traveling by train because"

In [11]:
content = prompt

with tps_report(max_tokens), Live(render(content)) as live:    
    for token in generator(prompt):
        content += token
        live.update(render(content))

Output()

Generated 30 tokens in 8.3 s (3.6 tps)


# Prompt 1: Paris

In [12]:
prompt = "Paris is an amazing place to visit,"

In [13]:
content = prompt

with tps_report(max_tokens), Live(render(content)) as live:    
    for token in generator(prompt):
        content += token
        live.update(render(content))

Output()

Generated 30 tokens in 6.0 s (5.0 tps)


# Prompt 2: Once Upon a Time

In [14]:
prompt = "Once upon a time"

In [15]:
content = prompt

with tps_report(max_tokens), Live(render(content)) as live:    
    for token in generator(prompt):
        content += token
        live.update(render(content))

Output()

Generated 30 tokens in 8.1 s (3.7 tps)


# Batched

In [17]:
prompts = (
    "I like traveling by train because",
    "Paris is an amazing place to visit,",
    "Once upon a time",
)

In [18]:
content = prompts

with tps_report(max_tokens), Live(render(content)) as live:    
    for tokens in generator(prompts):
        content = [content[i] + token for i, token in enumerate(tokens)]
        live.update(render(content))

Output()

Generated 30 tokens in 38.9 s (0.8 tps)
